In [2]:
!pip3 install scikit-learn
!pip3 install pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [146]:
import pandas as pd

input_file = "cardio_train.csv"
df = pd.read_csv(input_file, header=0)

In [169]:
from sklearn.preprocessing import OneHotEncoder 

df_cat = df.select_dtypes(include='object')
encoder = OneHotEncoder(sparse=False, handle_unknown='error')
df_cat_encoded = encoder.fit_transform(df_cat)
categorical_columns = [f'{col}_{cat}' for i, col in enumerate(df_cat.columns) for cat in encoder.categories_[i]]
df_num = df.select_dtypes(exclude='object')
df_cat = pd.DataFrame(df_cat_encoded, columns=categorical_columns)
df = df_num.join(df_cat)

,id,age,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,gender_man,gender_women,above_above,above_normal,above_well above,gluc_above,gluc_normal,gluc_well above
0,0,18393,168,62.0,110,80,0,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,20228,156,85.0,140,90,0,0,1,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2,18857,165,64.0,130,70,0,0,0,1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,3,17623,169,82.0,150,100,0,0,1,1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,4,17474,156,56.0,100,60,0,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,168,76.0,120,80,1,0,1,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
69996,99995,22601,158,126.0,140,90,0,0,1,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
69997,99996,19066,183,105.0,180,90,0,1,0,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
69998,99998,22431,163,72.0,135,80,0,0,0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [189]:
Y = df[["cardio"]]
X = df.loc[:, df.columns != 'cardio']
fold_size = 20
print("std for each column is for a specific fold:")
print(X[0:fold_size].std())
print("mean for each column is for a specific fold:")
print(X[0:fold_size].mean())

std for each column is for a specific fold:
id                     9.599342
age                 2465.906693
height                 8.243658
weight                15.932340
ap_hi                 12.763022
ap_lo                 10.310955
smoke                  0.307794
alco                   0.223607
active                 0.502625
gender_man             0.502625
gender_women           0.502625
above_above            0.223607
above_normal           0.444262
above_well above       0.410391
gluc_above             0.223607
gluc_normal            0.366348
gluc_well above        0.307794
dtype: float64
mean for each column is for a specific fold:
id                     14.60
age                 19061.35
height                164.20
weight                 73.95
ap_hi                 120.50
ap_lo                  77.00
smoke                   0.10
alco                    0.05
active                  0.60
gender_man              0.40
gender_women            0.60
above_above             0.05
abov

In [190]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()
y_train = y_train.ravel()
y_test = y_test.ravel()

In [204]:
from sklearn.model_selection import KFold, GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import GaussianNB

best_kf = 2
best_kf_value = 0 
best_st = 2
best_st_value = 0
model = GaussianNB()

for k in range(2, 72, 10):
    kf = KFold(n_splits=k)
    gs = GridSearchCV(model, cv=kf, param_grid={}, return_train_score=True)
    gs.fit(x_train, y_train)
    if gs.cv_results_['mean_test_score'][0] >= best_kf_value: 
        best_kf_value = gs.cv_results_['mean_test_score'][0]
        best_kf = k 
    
    stf = StratifiedKFold(n_splits=k)
    gs = GridSearchCV(model, cv=stf, param_grid={}, return_train_score=True)
    gs.fit(x_train, y_train)
    if gs.cv_results_['mean_test_score'][0] >= best_st_value: 
        best_st_value = gs.cv_results_['mean_test_score'][0]
        best_st = k
        
print("Best k fold:", best_kf)
print("Accuracy of best k fold:", best_kf_value)
print("Best stratified fold:", best_st)
print("Accuracy of best stratified fold:", best_st_value)

2
0.5520535714285715
2
0.5487857142857142


In [203]:
from sklearn.model_selection import cross_val_score

best_kf_cv = KFold(n_splits=best_kf) if best_st_value > best_kf_value else StratifiedKFold(n_splits=best_st)
model = GaussianNB()
gs = GridSearchCV(model, cv=best_kf_cv, param_grid={}, return_train_score=True)
gs.fit(x_train, y_train)
gs.cv_results_

{'mean_fit_time': array([0.0122329]),
 'std_fit_time': array([0.00302231]),
 'mean_score_time': array([0.00890172]),
 'std_score_time': array([0.00196826]),
 'params': [{}],
 'split0_test_score': array([0.56239286]),
 'split1_test_score': array([0.54171429]),
 'mean_test_score': array([0.55205357]),
 'std_test_score': array([0.01033929]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([0.56760714]),
 'split1_train_score': array([0.54207143]),
 'mean_train_score': array([0.55483929]),
 'std_train_score': array([0.01276786])}

In [213]:
from sklearn.metrics import accuracy_score

best_model = gs.best_estimator_
y_pred = best_model.predict(x_test)

print("accuracy of model is:", accuracy_score(y_test, y_pred))

(14000,)
(14000,)
              precision    recall  f1-score   support

           0       0.51      0.97      0.67      6917
           1       0.78      0.10      0.17      7083

    accuracy                           0.53     14000
   macro avg       0.64      0.53      0.42     14000
weighted avg       0.65      0.53      0.42     14000

accuracy of model is:  0.5287142857142857
